In [7]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC

### Week 4 Work

In [8]:
data = pd.read_parquet("./dsc-capstone-2022/DSC180B.parquet")
data = data[["amount", "dt", "new_category"]]
data

FileNotFoundError: [Errno 2] No such file or directory: './dsc-capstone-2022/DSC180B.parquet'

In [ ]:
data.dtypes

In [ ]:
data['new_category'].value_counts()

In [ ]:
# step 1: try to make a model without the text column

In [ ]:
data['year'] = data['dt'].dt.year # ranges 2010 - 2022
data['month'] = data['dt'].dt.month # ranges 1 - 12
data['is_holiday'] = data['month'].apply(lambda x: 1 if x == 11 or x == 12 else 0) # 0 or 1
data['day'] = data['dt'].dt.dayofweek # to be discarded
data['is_weekend'] = data['day'].apply(lambda x: 1 if x == 5 or x == 6 else 0) # 0 or 1

# amount: consider encoding whole numbers
data['is_whole_number'] = data['amount'].apply(lambda x: 1 if str(x).split(".")[1] == "0" else 0)
data

In [ ]:
y_test = np.array(y_test['new_category'])
accuracy = (preds == y_test).mean()
accuracy2 = (preds2 == y_test).mean()
print("Logistic Regression accuracy: {}, \nSGDClassifier accuracy: {}".format(accuracy, accuracy2)) # accuracy is for LogRes, accuracy2 is for SGDClassifier

### EDA

In [5]:
data['day'].value_counts()
# interesting -- most transactions are from monday and very few are from sat or sun

NameError: name 'data' is not defined

In [6]:
data['year'].value_counts().sort_index()
# mostly from 2017-2022

NameError: name 'data' is not defined

### Checking if there is a difference between November/December and rest of year

In [1]:
holidays = data[data['month'] >= 11]
not_holidays = data[data['month'] < 11]
print(len(holidays) + len(not_holidays) == len(data)) # proves proper split
holidays_vc = holidays['new_category'].value_counts(normalize=True, sort=False).sort_index()
not_holidays_vc = not_holidays['new_category'].value_counts(normalize=True, sort=False).sort_index()

X = np.arange(8)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, holidays_vc, color = 'b', width = 0.25)
ax.bar(X + 0.25, not_holidays_vc, color = 'g', width = 0.25)
plt.xticks(np.arange(8), holidays_vc.index, color='black', rotation=45, fontsize='16', horizontalalignment='right')
plt.legend(labels = ["holidays", "not holidays"])
plt.show()

NameError: name 'data' is not defined

In [2]:
holidays = data[data['month'] > 11]
not_holidays = data[data['month'] <= 11]
print(len(holidays) + len(not_holidays) == len(data)) # proves proper split
holidays_vc = holidays['new_category'].value_counts(normalize=True, sort=False).sort_index()
not_holidays_vc = not_holidays['new_category'].value_counts(normalize=True, sort=False).sort_index()

X = np.arange(8)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, holidays_vc, color = 'b', width = 0.25)
ax.bar(X + 0.25, not_holidays_vc, color = 'g', width = 0.25)
plt.xticks(np.arange(8), holidays_vc.index, color='black', rotation=45, fontsize='16', horizontalalignment='right')
plt.legend(labels = ["holidays", "not holidays"])
plt.show()

NameError: name 'data' is not defined